In [7]:
# 필요 패키지 임포드
import pandas
import numpy#
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA,NMF
import matplotlib.pyplot as plt
from numpy.linalg import eigh
import seaborn

#
# 1. DATA LOAD
# 
# Kaggle에서 제공하는 [BNP Paribas Cardif Claims Management](https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/data)데이터를 활용
# BNP Paribas Cardif 청구관리 데이터 셋으로 고객의 클레임 여부를 판단하는 데이터셋의 Subset
DATA_PATH = "c:\\Users\\WAI\\OneDrive\\03.WORKSPACE\\00.DATA\\"
df = pandas.read_csv(DATA_PATH + "BookExample.csv")
print(df.shape) # (20000, 33)
target_df = df[["target"]]
df.set_index(["ID","target"], inplace=True)

(20000, 33)


In [5]:
# NMF(non-negative matrix factorization) 비음수 행렬 분해
# 음수가 아닌 특성 행렬에 대한 차원 축소를 하는 방법입니다
# 비지도 학습 기법으로 샘플과 특성 사이에 잠재되어 있는 관계를 표현하는 행렬로 특성 행렬을 분해
# 음수의 값을 가지면 안되기 때문에 정규화를 통해 데이터를 0과 1사이로 변환해주도록 하겠습니다.

# NMF 알고리즘은 주로 시각철, 문서분류, 음파 분석등의 비정형 데이터의 차원을 축소하는 방법으로 사용됩니다.
# scikit learn의 NMF API를 사용하는 방법만 알아보도록 하겠습니다. 

# 범주화 및 인코딩 하기전의 데이터 셋
n_obj_cols = []
for col, type in zip(df.dtypes.index, df.dtypes.values):
    if type != ("object"):
        n_obj_cols.append(col)

n_obj_df = df[n_obj_cols]
print(n_obj_df.shape)

# PCA를 하기전에 Scale 조정을 통해 각 특성의 분산량을 동일 하게 수정
x = n_obj_df.values
x = MinMaxScaler().fit_transform(x)
features = n_obj_cols
#print(pandas.DataFrame(x, columns = features).head())
print(pandas.DataFrame(x, columns = features).describe())

(20000, 26)
             v1        v2        v4        v5            v6            v7  \
0  1.046932e-01  0.464949  0.331398  0.395763  3.574007e-01  3.574007e-01   
1  7.837838e-08  0.000000  0.000000  0.459563  8.401249e-08  1.065375e-07   
2  7.397962e-02  0.282890  0.372807  0.266308  5.471939e-01  4.419643e-01   
3  6.250014e-02  0.442429  0.357168  0.581372  2.884339e-01  2.235993e-01   
4  7.837838e-08  0.000000  0.000000  0.000000  8.401249e-08  1.065375e-07   

             v8            v9       v10           v11  ...       v18  \
0  6.471121e-04  5.000000e-01  0.027155  8.217054e-01  ...  0.005336   
1  1.150816e-01  4.915000e-08  0.070838  6.800000e-09  ...  0.000000   
2  9.822954e-04  6.333334e-01  0.041322  7.378049e-01  ...  0.012227   
3  8.597373e-03  4.482758e-01  0.353011  8.173741e-01  ...  0.061206   
4  3.890000e-08  4.915000e-08  0.056671  6.800000e-09  ...  0.000000   

        v19       v20       v21           v23       v25           v26  \
0  0.047148  0.9434

In [14]:
# NMF(non-negative matrix factorization)
nmf = NMF(n_components=5, random_state=1)
nmfComponents = nmf.fit_transform(x)
nmf_df = pandas.DataFrame(data=nmfComponents, columns = ['pcaComp1', 'pcaComp2', 'pcaComp3', 'pcaComp4', 'pcaComp5'])
print(nmf_df.head())

# 결과 출력
print("원본 특성 개수:", df.shape[1]) 
print("줄어든 특성 개수:", nmfComponents.shape[1])

# H 특성 행렬
print(nmf.components_.shape)
# 모두 양수 인지 확인
print(numpy.all(nmf.components_ >=0))

C:\Users\WAI\Anaconda3\envs\analysis\lib\site-packages\sklearn\decomposition\_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


   pcaComp1  pcaComp2  pcaComp3  pcaComp4  pcaComp5
0  0.093748  0.043048  0.003939  0.000000  0.066862
1  0.000000  0.101007  0.000000  0.048258  0.000000
2  0.103772  0.019344  0.000000  0.000000  0.055422
3  0.118763  0.055480  0.000000  0.008671  0.047783
4  0.000000  0.096432  0.000000  0.000000  0.000000
원본 특성 개수: 31
줄어든 특성 개수: 5
(5, 26)
True


C:\Users\WAI\Anaconda3\envs\analysis\lib\site-packages\sklearn\decomposition\_nmf.py:1091: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)
